In [ ]:
#|default_exp covalent_api

https://www.covalenthq.com/docs/api/balances/get-historical-token-balances-for-address/

**Functionality of `Covalent_Api`:**


**`get_historical_balances`**

Used to fetch the historical native, fungible (ERC20), and non-fungible (ERC721 & ERC1155) tokens held by an address at a given block height or date. Response includes daily prices and other metadata.

**`get_token_holders`**

Get token holders as of any block height (v2)

Commonly used to get a list of all the token holders for a specified ERC20 or ERC721 token. Returns historic token holders when block-height is set (defaults to latest). Useful for building pie charts of token holders.


**`get_holders_portfolios`**

Get coins held by an address.


**Other functionality of notebook:**

**`union_top_n`**

Computes all the coins in the top n portfolios of a token (i.e. union of all portfolios

**`intersection_count`**

 For each token in the union, count how many holders have that token in their portfolio.

**`Address_Holder_Data`**

 Wrapper to compute intersection_count

**`contract_name_if_k_holders`**

Finds contracts with exactly k holders in intersection_dict

**`contract_name_if_more_than_k_holders`**

Finds contracts with more than k holders in intersection_dict


In [4]:
#| export

import json
import requests
from requests.auth import HTTPBasicAuth
from requests.exceptions import HTTPError, RequestException
import time


from fastcore.basics import *



In [5]:
#| hide
from fastcore.test import *


In [6]:
#| export

def wei_to_eth(wei):
    "Simple helper function to convert wei to eth"
    return wei / 10**18

In [7]:
#| export

import logging

# Set up basic logging configuration
logging.basicConfig(level=logging.WARNING, format='%(message)s')

logger = logging.getLogger(__name__)

In [52]:
#| export

#The general API is: 
#                       def some_function(self,args):
#                            url = ... #define the url using args
#                            return self.get_items(url)

#rather than writing self.get_items(url) every time, we can use a decorator to do this for us.

def url_decorator(func):
    def wrapper(self, *args, **kwargs):
        url = func(self, *args, **kwargs)
        return self.get_items(url)
    return wrapper

class Covalent_Api:
    """This class is used to interact with the Covalent API.
    """

    def __init__(self, api_key='cqt_rQ37X8PXHqGxkycgdXHJtkFhfwyP'):
        self.api_key = api_key


    #TODO: understand better
    def get_items(self, url, retries=3, delay=1):
        """Given a url, get the items from the API.
        Inputs:
            `url`, a string, the url to get the items from.
            `retries`, an integer, the number of times to retry the request.
            `delay`, an integer, the number of seconds to wait between retries.

        Outputs:
            `items`, a list of dictionaries, each dictionary containing information about a token/address/etc
        """
        headers = {"accept": "application/json"}
        basic = HTTPBasicAuth(f'{self.api_key}', '')

        while retries > 0:
            try:
                response = requests.get(url, headers=headers, auth=basic)
                response.raise_for_status()  # Check for HTTP errors
                data = response.json()

                # Access 'data' field
                data = data.get('data')
                if data is None:
                    return None

                # Check if data is a list
                if type(data) is list:
                    return data

                # Access 'items' field
                return data.get('items')
            
            except HTTPError as e:
                if e.response.status_code == 429:  # HTTP Status Code for 'Too Many Requests'
                    time.sleep(delay)  # Adjust wait time
                    retries -= 1
                else:
                    return None

            except RequestException as e:
                return None


    @url_decorator
    def get_historical_balances(self,chainName:str,walletAddress:str,date:str, quote_currency="USD")->list:
        """Given a wallet address, get the historical balances for that wallet address on the specified chain and date.
            Inputs:
                `chainName`, e.g. 'eth-mainnet'
                `walletAddress`, e.g. '0x7364a0f792e073814B426c918bf72792575b6c18'
                `date`, e.g. '2021-01-01'.
                `quote_currency`, e.g. 'USD'
            Outputs:
                `items`, a list of dictionaries, each dictionary containing the balance of a token at a given date along with some additional metadata.
            Note:
                While the core function generates a URL, the applied @url_decorator 
                modifies the return behavior to fetch items using that URL.
        """
        assert type(date) is str, "date must be a string"

        url = f"https://api.covalenthq.com/v1/{chainName}/address/{walletAddress}/historical_balances/?quote-currency={quote_currency}&date={date}"

        return url 
    
    @staticmethod
    def print_balance(items):
        "Helper function to print the balance out, i.e. the object returned by `get_historical_balances`"

        for item in items:
            print(f"{item['contract_name']} balance: {item['balance']}\n")
    
    @url_decorator
    def get_token_holders(self, chainName: str, tokenAddress: str, block_height=None, page_size=100, page_number=None) -> list:
        """
        Fetches the token holders for a specific token on a given chain.
        Inputs:
            `chainName`: The chain name e.g. 'eth-mainnet'.
            `tokenAddress`: The token's address.
            `block_height`: Ending block to define a block range.
            `page_size`: Number of items per page. Supported values are 100 and 1000.
            `page_number`: 0-indexed page number to begin pagination.
        Outputs:
            `items`, a list of dictionaries, each dictionary containing information about a token holder.

        Note: This gets the holders by percentage (i.e. largest holdest to smallest)

        Note:
        While the core function generates a URL, the applied @url_decorator 
        modifies the return behavior to fetch items using that URL.
        """

        base_url = f"https://api.covalenthq.com/v1/{chainName}/tokens/{tokenAddress}/token_holders_v2/?"
        
        if block_height is not None:
            base_url += f"&block-height={block_height}"
        if page_size is not None:
            base_url += f"&page-size={page_size}"
        if page_number is not None:
            base_url += f"&page-number={page_number}"

        return base_url

    def get_holders_portfolios(self,n_lst,chainName:str,date:str,quote_currency="USD",log_output=False):
        """Input: 
                `n_lst`: a list of dictionaries containing the top holders of a token. Dictionary must contain the key 'address'.
                 See get_historical_balances for description of other inputs.
           Output: the same list of dictionaries with the portfolio added.
        """

        # Setting up logging level based on the log_output value
        if log_output:
            logger.setLevel(logging.INFO)
        else:
            logger.setLevel(logging.WARNING)

        #loop over the list of dictionaries (holders of the token)
        for _holder in n_lst: #this loop should be parallelized, but ok for now provided it isn't too big.

            _address = _holder['address']
            _items = self.get_historical_balances(chainName=chainName,walletAddress=_address, quote_currency=quote_currency, date=date)
            #_items is a list of dictionaries

            #get the sum of the portfolio

            #TODO: Make this more efficient: possibly convert to numpy array or dataframe or something and then sum
            none_to_zero = lambda x: 0 if x is None else x
            portfolio_sum = sum([none_to_zero(holding['quote']) for holding in _items])
            
            _holder['portfolio']=_items #update the _holder with the whole portfolio
            _holder['portfolio_sum']=portfolio_sum #update the _holder with the total value of the portfolio

            logger.info(f'got {_address} portfolio')  # Logging statement in place of print
            
        return n_lst
    
    #It seems that we already have this data via `get_historical_balances` throughts ['items']['quote'] #usd value
    # @url_decorator
    # def get_prices(self,chainName:str,address:str,quote_currency="USD",dates=None):
    #     """Input: 
    #             `chainName`: e.g. 'eth-mainnet'
    #             `address`: e.g. '0x7364a0f792e073814B426c918bf72792575b6c18'
    #             `quote_currency`: e.g. 'USD'
    #             `dates`: e.g. ['2021-01-01','2021-01-10'], i.e. from,to
    #        Output: 
    #     """

    #     if len(dates)==1:
    #         dates.append(dates[0]) #if only one date is specified, then we get the price for that date only.

    #     url = f"https://api.covalenthq.com/v1/pricing/historical_by_addresses_v2/{chainName}/{quote_currency}/{address}/?from={dates[0]}&to={dates[1]}"
    #     return url
    
    @staticmethod
    def print_prices(items):
        "Helper function to print the prices out i.e. `items` as returned by `get_prices`"
        print(f"Printing out prices of: {items[0]['contract_name']}")
        for k in items[0]['prices']:
            print(f"On {k['date']}, the price was {k['price']}, and the `pretty_price` was {k['pretty_price']}")


if __name__ == "__main__":


    #Debugging:

    #_address = XDOGE_holders[0]['address']
    #_items= cov_api.get_historical_balances(chainName=chainName,walletAddress=_address, quote_currency="USD", date="2023-08-30")

    n_lst = [{'address': XDOGE_holders[0]['address']}] #List of dictionaries
    n_lst = cov_api.get_holders_portfolios(n_lst,chainName,quote_currency,date="2023-08-30",log_output=False)

    print(n_lst[0]['portfolio_sum'])


2604501.761579631


Now we can combine `get_token_holders` and `get_historical_balances` to do things like the following:

- Given $m$ token addresses, compute the top $n$ token holders (one definition of "whale").
- Compute the historical balances of these token holders, for the present (say).
- Then we can do some analysis on the $m \times n$ addresses.

Example with $m=2$: say $BITCOIN and $CBOT are trending tokens, as they are this week. Then we can get the wallet information of the top 10 holders of each. So $m \times n = 10 \times 2 = 20 $ addresses in total. There may be several things of interest here. For example, what (if any) is the most common token in the 20 addresses outside of $BITCOIN and $CBOT. If all the top holders of these tokens also hold some small token that hasn't pumped yet, then this is relevant information. Let's build on this first idea now:

Step 1) Get top 10 token holders of two addresses:

In [10]:
nickcage = '0xfcaf0e4498e78d65526a507360f755178b804ba8'
hpbitcoin = '0x72e4f9F808C49A2a61dE9C5896298920Dc4EEEa9'

cov_api = Covalent_Api() #might need to pass an API key (different to default)

#List of top holders of nickcage and hpbitcoin. 
token_holders_nickcage = cov_api.get_token_holders(chainName='eth-mainnet',tokenAddress=nickcage,page_size=100,page_number=0)
token_holders_hpbitcoin = cov_api.get_token_holders(chainName='eth-mainnet',tokenAddress=hpbitcoin,page_size=100,page_number=0)

top_10_nickcage = [token_holders_nickcage[i] for i in range(10)]
top_10_hpbitcoin = [token_holders_hpbitcoin[i] for i in range(10)]

Step 2) Get the present wallet information of these "whales". i.e. update `top_10_.` with the respective portfolios:

In [11]:
top_10_nickcage = cov_api.get_holders_portfolios(n_lst=top_10_nickcage,chainName='eth-mainnet')
top_10_hpbitcoin = cov_api.get_holders_portfolios(n_lst=top_10_hpbitcoin,chainName='eth-mainnet')

Now we can do some analysis!

Step 3) Get union of portfolios:

In [12]:
#| export

def union_top_n(top_n):
    "Computes all the coins in the top n portfolios of a token (i.e. union of all portfolios))"

    top_n = list(set(
        (item['contract_name'],item['contract_address'])
        for entry in top_n #Basically: for each address in top_10_nickcage get all the tokens in their portfolio
        for item in entry['portfolio']
                    ))
    return top_n

In [13]:
union_top_10_nickcage = union_top_n(top_10_nickcage) #i.e. all the tokens help by the top 10 nickcage holders
union_top_10_hpbitcoin = union_top_n(top_10_hpbitcoin)#i.e. all the tokens help by the top 10 nickcage holders

Step 4): Get intersection counts:

In [14]:
#| export

def intersection_count(top_n:list, union_lst:list) -> dict:
    """
    For each token in the union, count how many holders have that token in their portfolio.
    Inputs:
        top_n: list of dictionaries, each dictionary is a holder
        union_lst: list of tokens, each token is a string
    Output:
        intersect_dict: dictionary, keys are tokens, values are integers
    """

    # Create a dictionary with default value as 0
    intersect_dict = {(token_name, token_address): 0 for token_name, token_address in union_lst}

    # Pre-compute contract addresses for each holder
    holder_portfolios = {}
    for holder in top_n:
        holder_portfolios[holder['address']] = set(item['contract_address'] for item in holder['portfolio'])

    # Update the intersection count
    for token_name, token_address in union_lst:
        for holder_address, contracts in holder_portfolios.items():
            if token_address in contracts:
                intersect_dict[(token_name, token_address)] += 1

    return intersect_dict



In [15]:
intersect_dict_top_10_nickcage = intersection_count(top_10_nickcage,union_top_10_nickcage)
intersect_dict_top_10_hpbitcoin = intersection_count(top_10_hpbitcoin,union_top_10_hpbitcoin)

And we can verify the following **tests**:
- that all holders amongst the top 10 of `nickcage` and `bitcoin` hold `nickcage` or `bitcoin` respectively (this must be vacuously true).
- that the counts are all larger than 1 (at least one address must own the token for it to have appeared in the union)

In [16]:
#names of the tokens
nickcage_name = top_10_nickcage[0]['contract_name']
hpbitcoin_name = top_10_hpbitcoin[0]['contract_name']
#addresses
nickcage_address = top_10_nickcage[0]['contract_address']
hpbitcoin_address = top_10_hpbitcoin[0]['contract_address']

#check that `nickcage` and `hpbitcoin` occur exactly 10 times in the top 10
test_eq(intersect_dict_top_10_nickcage[(nickcage_name,nickcage_address)],10)
test_eq(intersect_dict_top_10_hpbitcoin[(hpbitcoin_name,hpbitcoin_address)],10)

#Check that each token appears at least once 
for k in intersect_dict_top_10_nickcage: assert intersect_dict_top_10_nickcage[k]>=1
for k in intersect_dict_top_10_hpbitcoin: assert intersect_dict_top_10_hpbitcoin[k]>=1

Ok, let's do some exploration of hpbitcoin, say:

The line if k>len(intersect_dict): raises a ValueError if k is greater than the length of the dictionary. However, this may not be necessary because:

The length of the dictionary doesn't directly correlate with the number of holders (k). The dictionary could have many keys (contracts) with a few holders, or vice versa.
If k is greater than the number of holders in any contract, the function will naturally return an empty list, which might be the expected behavior.

In [17]:
#| export

def contract_name_if_k_holders(intersect_dict,k):
    "Get the names of the contracts that have exactly k holders"
    if k>len(intersect_dict):
        raise ValueError("k is greater than length of intersect_dict")
    return [contract_name for contract_name in intersect_dict.keys() if intersect_dict[contract_name] == k]

def contract_name_if_more_than_k_holders(intersect_dict,k):
    "Get the names of the contracts that have more than k holders"
    if k>len(intersect_dict):
        raise ValueError("k is greater than length of intersect_dict")
    return [contract_name for contract_name in intersect_dict.keys() if intersect_dict[contract_name] > k]

In [53]:
#| export

def contract_address_to_holders(n_holders,contract_address):
    "#Given an address, get the holders that hold that address"
    lst=[]
    for _holder in n_holders:
        for _token in _holder['portfolio']:
            if _token['contract_address'] == contract_address:
                lst.append(_holder)

    return lst

In [18]:
top_20_hpbitcoin = [token_holders_hpbitcoin[i] for i in range(20)]
#get portfolio of top 20 holders of HPB token
top_20_hpbitcoin = cov_api.get_holders_portfolios(n_lst=top_20_hpbitcoin,chainName='eth-mainnet')
#Get union
union_top_20_hpbitcoin = union_top_n(top_20_hpbitcoin) #i.e. all the tokens help by the top 100

#Get intersection
intersect_dict_top_20_hpbitcoin = intersection_count(top_n=top_20_hpbitcoin,union_lst=union_top_20_hpbitcoin)


In [19]:
contract_name_if_more_than_k_holders(intersect_dict_top_20_hpbitcoin,15)
#contract_name_if_k_holders(intersect_dict_top_20_hpbitcoin,18)

[('Rog Coin', '0x76c8fc20045a423f800cbb23ad9ba853eb85688a'),
 ('ETHEREUM2.0', '0xea498670e8de236c17d75c0bd09dd4b1b6f39eb2'),
 ('Russian Roulette', '0x2e24be72c0bdf09f63ed5217b01bd5ad1e98b2f6'),
 ('HarryPotterObamaSonic10Inu', '0x72e4f9f808c49a2a61de9c5896298920dc4eeea9'),
 ('Fino', '0x96a3b1227fdbe7342d1358ae4ed87adc4584d9cf'),
 ('Bitlord', '0x781bd109834c534dc0f799afdf65e6eb5151b839'),
 ('HarryPotterObamaSonic10Inu (BITCOIN2.0)',
  '0x464af6d0d292a61c1b71251a3d2354996e661153'),
 ('Friend Tech', '0x88c6aefe66fc619d073d3560bb5dc4d8a38f3e8c'),
 ('XDOGE', '0xd2b274cfbf9534f56b59ad0fb7e645e0354f4941'),
 ('Pond0x V2', '0x16148f8cdca3d703821f4a1a87123790c5e7459b'),
 ('Fuck Bald', '0x80267e32be79b4337a7cc861df97714440586aa7'),
 ('DFI.money2.0', '0xbc581118802bec35e755f0a85f62af6736a2c4a1'),
 ('Pond0x 2.0', '0x9a25b66c3f9c571074052456351990958bb80f3f'),
 ('Inu10SonicObamaPotterHarry', '0x46fad82e4c96ef5821eb4ae9f27fbc450ec472ee'),
 ('EpsteinClintonQanonPizzaGate10Inu',
  '0x682a21d52451bb93e4b

- 18 of the top 20 holders (*at present*) of HPbitcoin also own Bald token - which is a scam.

- Interestingly, at present 18/20 also hold `XDOGE`. Inspection of the `XDOGE` chart shows that it looks reasonably good. Market cap ~ 500k at present.

We can collect the `intersect_dict`, and other data through a simple wrapper:

In [20]:
#| export

class Address_Holder_Data:
    "wrapper class to get `intersection_count` for top n holders of a token (address)"

    def __init__(self, cov_api, tokenAddress, chainName,date,k,quote_currency="USD"): 
        store_attr()
        assert k <= 100, f"The value of k should be <= 100. The current value is {k}."
        
        self.intersect_dict_top_n = self.get_data()
        
    def get_data(self):
        self.token_holders = self.cov_api.get_token_holders(chainName=self.chainName, tokenAddress=self.tokenAddress, page_size=100, page_number=0)
        self.top_n_holders = [self.token_holders[i] for i in range(self.k)]
        self.top_n_holders = self.cov_api.get_holders_portfolios(n_lst=self.top_n_holders,chainName=self.chainName,quote_currency=self.quote_currency,date=self.date)
        self.union_top_n = union_top_n(self.top_n_holders)
        self.intersect_dict_top_n = intersection_count(top_n=self.top_n_holders, union_lst=self.union_top_n)

        return self.intersect_dict_top_n


In [55]:
#| export
from datetime import datetime
def todays_date():
    "Returns today's date as a string in UTC"
    return datetime.utcnow().strftime('%Y-%m-%d')


2023-08-30


Let's use this API to have a look at `XDOGE`:

In [347]:
xdoge_address = '0xd2b274cfbf9534f56b59ad0fb7e645e0354f4941'
xdoge_data = Address_Holder_Data(cov_api=cov_api, tokenAddress=xdoge_address, chainName='eth-mainnet', k=20)

contract_name_if_more_than_k_holders(xdoge_data.intersect_dict_top_n,k=10)

[('XDOGE', '0xd2b274cfbf9534f56b59ad0fb7e645e0354f4941'),
 ('Ether', '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee'),
 ('KuKu', '0xc3071803b9d23460820b516673fd3cec0415d0ed')]

We can also use covalent API to get historical prices of token:

In [32]:
items = cov_api.get_prices(chainName='eth-mainnet',address=hpbitcoin,quote_currency="USD",dates=["2023-08-25","2023-08-26"])
cov_api.print_prices(items)

Printing out prices of: HarryPotterObamaSonic10Inu
On 2023-08-26, the price was 0.0918059, and the `pretty_price` was $0.09
On 2023-08-25, the price was 0.104179405, and the `pretty_price` was $0.10


In [448]:
items = cov_api.get_prices(chainName='eth-mainnet',address=hpbitcoin,quote_currency="USD",dates=["2023-08-25","2023-08-26"])
print(f"Printing out prices of: {items[0]['contract_name']}")
cov_api.print_prices(items)

'HarryPotterObamaSonic10Inu'

In [59]:
import nbdev
nbdev.export.nb_export('covalent_api.ipynb', './')